In [3]:
%cd src

d:\Repos\Other\toolformer\backend\src


In [4]:
import mic
import asyncio
import requests

d:\Repos\Other\toolformer\backend\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
stop_future: asyncio.Future = asyncio.get_running_loop().create_future()
voice_queue=await mic.start_background(stop_future)

[MIC] Starting listener
[MIC] found microphone
[MIC] Starting transcriber


In [6]:
# await voice_queue.get()

LLM inference is delegated to LLaMA-Adapter running on localhost
https://github.com/TeamDman/LLaMA-Adapter

In [7]:
async def get_response(human_input):
    url = "http://127.0.0.1:5000/"
#     prompt = f"""The following is a conversation between a human and an AI.
# Human: {human_input}
# AI:"""
    prompt=f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{human_input}

### Response:"""
    data = {"prompts": [prompt]}
    response = requests.post(url, json=data).json()[0]
    response = response.replace(prompt, "")
    if "Human:" in response:
        response = response.split("Human:")[0]
    response = response.strip()
    return response
# await get_response("Hi there!")

In [8]:
from TTS.api import TTS
from IPython.display import Audio, display
model_name = TTS.list_models()[20]
print(model_name)
tts = TTS(model_name, gpu=True)
print(tts.speakers)
print("speakers", tts.speakers)

tts_models/en/blizzard2013/capacitron-t2-c50
 > tts_models/en/blizzard2013/capacitron-t2-c50 is already downloaded.
 > vocoder_models/en/blizzard2013/hifigan_v2 is already downloaded.
 > Using model: tacotron2
 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:80.0
 | > mel_fmax:12000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:25.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 2

In [9]:
def speak(text):
    wav = tts.tts(text, speaker=None)
    # display(Audio(wav, rate=tts.synthesizer.output_sample_rate))
    tts.synthesizer.save_wav(wav=wav, path="output.wav")
    import winsound
    winsound.PlaySound("output.wav", winsound.SND_FILENAME)
# speak("Hello there")

In [10]:
def empty_queue(q: asyncio.Queue):
  while not q.empty():
    # Depending on your program, you may want to
    # catch QueueEmpty
    q.get_nowait()
    q.task_done()

In [11]:
empty_queue(voice_queue)


In [18]:
from pprint import pprint
empty_queue(voice_queue)

# while True:
human = (await voice_queue.get()).strip()
if human != "":
    pprint(f"Human: {human}")
    response = await get_response(human)
    pprint(f"AI: {response}")
    speak(response)

('Human: How do I, myself, my wife and Bigfoot, get water out of our ear and '
 'put it into the ocean? What sauce do you recommend for this?')
('AI: To get water out of your ear, you can use a bulb syringe or a cotton '
 'swab to gently clean out the ear canal. To put the water into the ocean, you '
 'can use a bucket or a watering can to collect the water and pour it into the '
 'ocean. For the sauce, you can use a mixture of lemon juice and salt to help '
 'clean the ear canal.')
 > Text splitted to sentences.
['To get water out of your ear, you can use a bulb syringe or a cotton swab to gently clean out the ear canal.', 'To put the water into the ocean, you can use a bucket or a watering can to collect the water and pour it into the ocean.', 'For the sauce, you can use a mixture of lemon juice and salt to help clean the ear canal.']
tə ɡɛt wˈɔːɾɚɹ ˌaʊɾəv jʊɹ ˈɪɹ, juː kæn jˈuːz ɐ bˈʌlb sɪɹˈɪndʒ ɔːɹ ɐ kˈɑːʔn̩ swˈɑːb tə dʒˈɛntli klˈiːn ˈaʊt ðɪ ˈɪɹ kənˈæl.
 [!] Character '̩' not found 